In [ ]:
import os
os.getcwd()

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras import optimizers



In [ ]:
#from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model 
#from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
img_width, img_height = 299, 299
train_data_dir = "/data/fgcvx/train/images/"
validation_data_dir = "/data/fgcvx/train/images/"
nb_train_samples = 81417
nb_validation_samples = 8343 

In [ ]:
nr_categories = 1394
batch_size = 16
epochs = 50

In [ ]:
# model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
base_model.output

In [ ]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(nr_categories, activation='softmax')(x)



In [ ]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])


In [ ]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, 
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        validation_split=0.1)

In [ ]:
img_width, img_height = 128, 128
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        #save_to_dir = "/data/fg/train",
        subset="training",
        shuffle=True, seed=13,
        class_mode='categorical')

In [ ]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        #save_to_dir = "/data/fg/val",
        batch_size=batch_size,
        shuffle=True, seed=13,
        class_mode='categorical')

In [ ]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("inception_fungi_features.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [ ]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
#import numpy as np
#validation_generator.class_indices

In [ ]:
len(os.listdir("/data/fgcvx/train/images/10000_Abortiporus_biennis"))

In [ ]:
import numpy as np

In [ ]:
train_labels = []
species = os.listdir("/data/fgcvx/train/images/")

for specie in species:
    n = len(os.listdir("/data/fgcvx/train/images/{}".format(specie)))
    train_labels.append([train_generator.class_indices[specie]]*n)
    
    

In [ ]:
val_labels = []
species = os.listdir("/data/fgcvx/train/images/")

for specie in species:
    n = len(os.listdir("/data/fgcvx/train/images/{}".format(specie)))
    val_labels.append([validation_generator.class_indices[specie]]*n)
    
    

In [ ]:
import itertools
val_labels_ = list(itertools.chain(*val_labels))
train_labels_ = list(itertools.chain(*train_labels))

In [ ]:
len(train_labels_)

In [ ]:
tr = np.array(train_labels_)

In [ ]:
vl = np.array(val_labels_)


In [ ]:
tr == vl

In [ ]:
bottleneck_features_train = model.predict_generator(train_generator)

In [ ]:
bottleneck_features_val = model.predict_generator(validation_generator)

In [ ]:
with open('/data/bottleneck_features_train_fg_inceptionv3.npy', 'wb') as fh:
    np.save(fh, str(bottleneck_features_train))
bottleneck_features_train.shape

In [ ]:
with open('/data/bottleneck_features_val_fg_inceptionv3.npy', 'wb') as fh:
    np.save(fh, str(bottleneck_features_val))
bottleneck_features_val.shape

In [ ]:
print(1)

In [ ]:
validation_data = bottleneck_features_val
train_data = bottleneck_features_train
#validation_labels = np.array([0] * 400 + [1] * 400)

In [ ]:
tr

In [ ]:
#train_data = np.load(open('/data/bottleneck_features_train_fg_inceptionv3.npy', "rb"))
# the features were saved in order, so recreating the labels is easy
#train_labels = np.array([0] * 1000 + [1] * 1000)
train_labels = tr
validation_labels = vl


In [ ]:
print(len(train_labels))
print(len(val_labels))

In [ ]:
train_data.shape[1:]
bottleneck_features_train.shape

In [ ]:
x = model.output
# x = Flatten()(x)

In [ ]:
x = Flatten()(x)
x.shape

In [ ]:
#train_data = np.load(open('/data/bottleneck_features_train_fg_inceptionv3.npy', "rb"))
# the features were saved in order, so recreating the labels is easy
#train_labels = np.array([0] * 1000 + [1] * 1000)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(nr_categories*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nr_categories, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model_fg.h5')



In [ ]:
model.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])